In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.utils import shuffle
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [2]:
if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("GPU is not available.")

GPU is available.


In [3]:
class CatDogDataset(Dataset):
    def __init__(self, cat_dir, dog_dir, transform=None):
        self.cat_dir = cat_dir
        self.dog_dir = dog_dir
        self.cat_imgs = [os.path.join(cat_dir, file) for file in os.listdir(cat_dir) if file.endswith('.jpg')]
        self.dog_imgs = [os.path.join(dog_dir, file) for file in os.listdir(dog_dir) if file.endswith('.jpg')]
        self.all_imgs = self.cat_imgs + self.dog_imgs
        self.labels = [0] * len(self.cat_imgs) + [1] * len(self.dog_imgs)
        self.transform = transform

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_path = self.all_imgs[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (100,100))
        img = img / 255.0
        labels = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        return img, labels

cat_dir = 'data/train/cat/'
dog_dir = 'data/train/dog/'

dataset = CatDogDataset(cat_dir, dog_dir)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
# cat_dir = 'data/train/cat/'
# dog_dir = 'data/train/dog/'

# dataset = CatDogDataset(cat_dir, dog_dir)

# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [16]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(18432, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)

        print(f"Shape after conv layers: {x.shape}")
        
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = SimpleCNN()

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
crtierion = nn.BCELoss
optimizer = optim.Adam(model.parameters(), lr=0.001)


cuda


In [20]:
num_epochs = 10

print(outputs.shape)
print(labels.shape)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = crtierion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader)}")

print("Training complete!")


torch.Size([32])
torch.Size([32])
Shape after conv layers: torch.Size([32, 18432])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [3]:
def load_images(image_path, target_size=(100,100)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img / 255.0

In [4]:
cats = 'data/train/cat/'
dogs = 'data/train/dog/'

cat_imgs = [load_images(cats + file) for file in os.listdir(cats) if file.endswith('.jpg')]
dog_imgs = [load_images(dogs + file) for file in os.listdir(dogs) if file.endswith('.jpg')]

cat_labels = [0] * len(cat_imgs)
dog_labels = [1] * len(dog_imgs)

In [5]:
x = np.array(cat_imgs + dog_imgs)
y = np.array(cat_labels + dog_labels)

x, y = shuffle(x, y)

print(f"Image data shape: {x.shape}")
print(f"Labels Shape: {y.shape}")

Image data shape: (275, 100, 100, 3)
Labels Shape: (275,)


In [24]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.datasets import ImageFolder

In [25]:
model = models.resnet18(pretrained=True)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
model = model.to(device)

/home/darshan/anaconda3/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/darshan/anaconda3/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
transofrm = transforms.Compose([
    transforms.Resize((224,244)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

train